In [45]:
###情感分析在3分类问题上
## 2019-11-23
## Liu,Yazhou

In [1]:
import pandas as pd 
import numpy as np 
import jieba
import multiprocessing
import re
import time 
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from sklearn.cross_validation import train_test_split
from tensorflow.python.keras.models import Sequential,model_from_yaml
from gensim.models import KeyedVectors
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional,Dropout,Activation
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import to_categorical

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 1.数据预处理 

### 1.1 添加labels

In [23]:
cate = {'neg':0,'neutral':1,'pos':2}
def add_lable(inputs,outputs,label):
    writors = open(outputs,'a',encoding = 'utf-8')
    with open(inputs,'r',encoding = 'utf-8') as file:
        for line in file.readlines():
            writors.write(str(label)+","+line)
# 将三个文件都添加标签并全部合并成Exampels_Labels.txt

add_lable('./data/neg.txt','./data/Exampels_Labels.txt',0)
add_lable('./data/neutral.txt','./data/Exampels_Labels.txt',1)
add_lable('./data/pos.txt','./data/Exampels_Labels.txt',2)

### 1.2 对全部的样本乱序

In [24]:
### 对原来的数据样本乱序 ### 
import random
def shuffles(inputs,outputs):
    contents = []
    file1 = open(inputs, 'r', encoding='utf-8') 
    file2 = open(outputs, 'w', encoding='utf-8')
    for line in file1.readlines():
        contents.append(line)
    
    random.shuffle(contents)
    for content in contents:
        file2.write(content)
    
    file1.close()
    file2.close()

# 样本乱序后存放在'./data/Exampels_Labels_random.txt'
shuffles('./data/Exampels_Labels.txt','./data/Exampels_Labels_random.txt')

In [36]:
def get_cn_model(): 
    cn_model = KeyedVectors.load_word2vec_format('embeddings/sgns.zhihu.bigram', 
                                             binary=False, unicode_errors="ignore")
    
    return cn_model
def get_train_data(texts_clean,texts_target,LENS = 1):
    path = './data/Exampels_Labels_random.txt'
    with open(path,'r',encoding = 'utf-8') as file:
        for line in file.readlines():
            items = line.split(',')
      
            text = items[1]
            text = re.sub('[^0-9A-Za-z\u4e00-\u9fa5]','',text)
            if len(text)>LENS and text!='\n':
                if items[0] == '0':
                    texts_target.append(0)
                elif items[0] == '1':
                    texts_target.append(1)  ##### train_target 这里一定要存放数字，不能是字符串'1'/'0'
                else:# items[0] == '2'
                    texts_target.append(2)
                texts_clean.append(text)
    return texts_target,texts_clean

def get_train_tokens(train_tokens,texts_clean,cn_model,LENS =1):
    for text in texts_clean:
        cut = jieba.cut(text)
        cut_list = [i for i in cut]
        for i, word in enumerate(cut_list):
            try:
                cut_list[i] = cn_model.vocab[word].index
            except KeyError:
                cut_list[i] = 0
        train_tokens.append(cut_list)
    return train_tokens

### 1.3 获取训练数据和tokens

In [37]:
cn_model = get_cn_model()
texts_clean  =[]
texts_target =[]
train_tokens =[]
texts_target,texts_clean = get_train_data(texts_clean,texts_target)
train_tokens             = get_train_tokens(train_tokens,texts_clean,cn_model)

In [39]:
len(texts_clean),len(texts_target),len(train_tokens)

(21056, 21056, 21056)

### 1.4将 text_labels and text_tokens 写入本地文件

In [40]:
writors_labels = open('./data/text_labels.txt','w',encoding = 'utf-8')
writors_text   = open('./data/texts_clean.txt','w',encoding = 'utf-8')
for label in texts_target:
    writors_labels.write(str(label)+'\n')

for line in texts_clean:
    writors_text.write(line+'\n')
    
writors_labels.close()
writors_text.close()

### 1.5 max_tokens and vocab_size 

In [29]:
def max_token(train_tokens):
    num_tokens = [ len(tokens) for tokens in train_tokens ]
    num_tokens = np.array(num_tokens)
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    max_tokens = int(max_tokens)
    return max_tokens

def vocab_size(train_tokens):
    vocab_tokens = []
    i = 0
    t1 = time.time()
    for tokens in train_tokens:
        for token in tokens:
            i+=1
            if token not in vocab_tokens:
                vocab_tokens.append(token)
            else:
                pass
        if i%(1e5) == 0:
            print("处理前%d的词花费的时间是:%0.2f"%(i,(time.time() - t1)/60),'mins')
    print("词汇表大小是：",len(vocab_tokens))
    return len(vocab_tokens)

In [7]:
max_tokens = max_token(train_tokens)
max_tokens

387

In [8]:
#num_words = vocab_size(train_tokens)
#num_words

### 1.6 通过词嵌入矩阵提取特征并padding 

In [30]:
def get_embedding_matrix(embedding_dim,num_words,cn_model):
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for i in range(num_words):
        embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]
    embedding_matrix = embedding_matrix.astype('float32')
    return embedding_matrix

def add_padding(train_tokens,train_target,max_tokens,num_words):
    train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')
    train_pad[train_pad>=num_words] = 0
    train_targets = np.array(train_target)
    return train_pad,train_targets
def model(y_train,y_test,X_train,X_test,epochs,batch_size,num_classes):#mask_zero = True
    model = Sequential()
    model.add(Embedding(num_words,
                        embedding_dim,
                        mask_zero= True,
                        weights=[embedding_matrix],
                        input_length=max_tokens,
                        trainable=False))
    model.add(Bidirectional(LSTM(units= 64, return_sequences=True)))
    model.add(LSTM(units=16, return_sequences=False))
    model.add(Dense(3, activation='softmax')) # Dense全连接层，输出维度是3
    #model.add(Activation('softmax'))
    optimizer = Adam(lr=1*1e-2)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    
    model.summary()
    path_checkpoint = './main_model/sentiment_checkpoint3cates.keras'
    checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)
    try:
        model.load_weights(path_checkpoint)
    except Exception as e:
        print(e)
        
    earlystopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    lr_reduction  = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-8, patience=0,
                                       verbose=1)
    callbacks = [earlystopping, checkpoint,lr_reduction]
    y_train_ = to_categorical(y_train,num_classes = num_classes)
    y_test_  = to_categorical(y_test,num_classes = num_classes)
    
    model.fit(X_train, y_train_,validation_split=0.2, epochs=epochs,
          batch_size=batch_size,
          callbacks=callbacks)
    from tensorflow.python.keras.models import save_model
    save_model(model,'./main_model/sentiment_checkpoint3cates.h5')
    result = model.evaluate(X_test, y_test_)
    print('Accuracy is :{0:.2%}'.format(result[1]))
    return result[1]*100

### 2.训练前数据准备

In [41]:
embedding_dim = 300
num_words     = 35799
max_tokens    = 387

embedding_matrix = get_embedding_matrix(embedding_dim,num_words,cn_model)
train_pad,texts_target_pad = add_padding(train_tokens,texts_target,max_tokens,num_words)
X_train, X_test, y_train, y_test = train_test_split(train_pad, # array
                                                    texts_target,  # array
                                                    test_size=0.2,
                                                    random_state=420)

In [32]:
len(train_tokens),len(texts_target),len(train_pad)

(21056, 21056, 21056)

### 3.开始训练Model

In [46]:
epochs      = 10
batch_size  = 512
num_classes = 3
max_tokens  = 387
accuracy = model(y_train,y_test,X_train,X_test,epochs,batch_size,num_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 387, 300)          10739700  
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         186880    
_________________________________________________________________
lstm_8 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 51        
Total params: 10,935,911
Trainable params: 196,211
Non-trainable params: 10,739,700
_________________________________________________________________
Train on 13475 samples, validate on 3369 samples
Epoch 1/20
13475/13475 [==============================] - 668s - loss: 0.3440 - acc: 0.8769 - val_loss: 0.3032 - val_acc: 0.8985
Epoch 2/20
13475/13475 [==============================] - 753s - loss: 0.2070 

KeyboardInterrupt: 